In [10]:
import time
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
from PIL import Image
from io import BytesIO


In [ ]:
import faiss
from transformers import CLIPProcessor, CLIPModel

In [11]:
def origin_run(goods_soup,img):
        # 카테고리
        category = goods_soup.find_all('p', attrs={'class': 'item_categories'})[0] # 대분류 이름
        category_detail = category.get_text().replace(' ', '').replace('\n', '').replace('(무신사스탠다드)', '').split('>') # 중분류 이름
        if(len(category.select('a')) <= 2):
            # continue
            return

        # 타이틀
        title_list.append(img['title'])
        url_list.append('https:' + img['href'])
        # class
        price = goods_soup.find_all('span', attrs={'class': 'product_article_price'})[0] # 가격 정보
        price_list.append(re.sub(r'[^0-9]', '', price.get_text().split()[-1]))
        
        # 이미지 URL 추출
        image_src = img.get('src')
        if not image_src.startswith('http:'):
            image_src = 'https:' + image_src
        
        # 이미지 다운로드
        response = requests.get(image_src)
        image = Image.open(BytesIO(response.content))
        display(image)
        # 이미지 잘 찍히는지 확인하고
        
        
        # 잘 찍히면 이미지 -> fashion clip으로 txt로 변환하고
        
        
        # txt-> 전체 저장소에 임베딩 faiss

    
        print(start, " finish")

In [12]:
# def crawl(start, title_list, url_list, category_list, price_list, gender_list):
#전역
def crawl(start):
    print(start, " start")

    url = ('https://www.musinsa.com/brands/musinsastandard?category3DepthCodes=&category2DepthCodes'
           '=&category1DepthCode=&colorCodes=&startPrice=&endPrice=&exclusiveYn=&includeSoldOut=&saleGoods'
           '=&timeSale=&includeKeywords=&sortCode=emt_high&tags=&page={'
           '}&size=90&listViewType=small&campaignCode=&groupSale=&outletGoods=false&boutiqueGoods=').format(
        start)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')

    for img in soup.find_all('a', attrs={'class': 'img-block'}):
        goods_num = img['href'].split('/')[-1]

        goods_url = 'https://www.musinsa.com/app/goods/{}'.format(goods_num)
        goods_response = requests.get(goods_url, headers={"User-Agent": "Chrome/39.0.2171.95"})
        goods_soup = BeautifulSoup(goods_response.text, 'lxml')

        # 상세 페이지 분류 시작
        origin_run(goods_soup,img)
        
    return


In [13]:
title_list=[]
url_list=[]
price_list=[]

In [14]:
crawl(1) 

1  start


AttributeError: 'NoneType' object has no attribute 'startswith'

In [7]:
for start in range(1,20):
    crawl(start) 

1  start


UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x000001532A7B24D0>

In [ ]:
df = pd.DataFrame({'상품명': title_list, 'url': url_list, '가격': price_list})
df.to_csv('무신사.csv', encoding='utf-8-sig')